In [1]:
source("./PhyloseqObjects.r")

Warning message in strsplit(conditionMessage(e), "\n"):
“input string 1 is invalid in this locale”

In [2]:
library('igraph')
packageVersion("igraph")

library("Hmisc")
packageVersion("Hmisc")

library("RColorBrewer")
packageVersion("RColorBrewer")

library("psych")
packageVersion("psych")

library("corrplot")
packageVersion("corrplot")


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



[1] ‘1.2.4.1’

Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units



[1] ‘4.2.0’

[1] ‘1.1.2’


Attaching package: ‘psych’

The following object is masked from ‘package:Hmisc’:

    describe

The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



[1] ‘1.8.12’

corrplot 0.84 loaded


[1] ‘0.84’

In [3]:
# getting phyloseq object: total 16S with raw counts, then split by material
physeq.whole

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 945 taxa and 27 samples ]
sample_data() Sample Data:       [ 27 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 945 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 945 tips and 943 internal nodes ]

In [4]:
physeq.whole.genus = tax_glom(physeq.whole, "Genus")

In [5]:
physeq.whole.genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 231 taxa and 27 samples ]
sample_data() Sample Data:       [ 27 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 231 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 231 tips and 230 internal nodes ]

In [6]:
# build a prevalence table for filtering
prevalencedf = apply(X = otu_table(physeq.whole.genus),
                     MARGIN = 1,
                     FUN = function(x){sum(x > 0)})

prevalencedf = data.frame(Prevalence = prevalencedf,
                          TotalAbundance = taxa_sums(physeq.whole.genus))

In [7]:
# filter out features that are not presnet in 10% of all samples
prevalenceThreshold = 0.10 * nsamples(physeq.whole.genus)
prevalenceThreshold

[1] 2.7

In [8]:
keepTaxa = rownames(prevalencedf)[(prevalencedf$Prevalence >= prevalenceThreshold)]
length(keepTaxa)
keepTaxa

[1] 23

[1] "ac09a700281e0e84b375bb5ae517a23e" "896c05c0ef629cc4725eb1d8db95f2e5"
 [3] "473b630262c9e46dc6f8f08886406faf" "4921e77373cf88d5d939898928db7f1d"
 [5] "af2a8d6190dac41427b5208bd79874e0" "04b867faf28bb8e9696ddab9bc9774a1"
 [7] "3112a0cc9050d0bee3e65ae6ac080099" "6ba2d1f0e3b971b2afeb348b44f26b2d"
 [9] "2424b82f393cce6b25e2f0fef86314f2" "a045fd276e32fb21f5c97e9c7068d226"
[11] "f508a05c1f9bbdaefd0ec0ff6f5db60b" "92eb684563a59988b1e9cc61e7c6eaa1"
[13] "157f6ba228fbb1d6d88977bcc75d3c63" "4c6043796448a2d50aaf470a409f36f7"
[15] "04f7a79aa778df478752a5e08bb5e68d" "7e2c443c0ab5d9675d2e14bf13cdbab5"
[17] "ba2bf0826df666849c91fc7512a57e78" "fa169ee0676b05a76e176e95deec2aa9"
[19] "82bc1e66ac43e7f68f9795809c4cad99" "0c00cc21bf4cb3fbc403e974cad42402"
[21] "d1589540ef4c33b985e1103a5e0921f9" "178902d35f055b7cc539fe7326a49c89"
[23] "8f1d6c5f6a03b21a67f492d2004679fa"

In [9]:
physeq.whole.genus.prevalent = prune_taxa(keepTaxa, physeq.whole.genus) 

In [10]:
#otu_table(physeq.whole.genus.prevelant)

## Relative

In [10]:
sample_sums(physeq.whole.genus.prevalent)

T1R1 T1R10  T1R4  T1R5  T1R7  T1R9  T2R1 T2R10  T2R4  T2R5  T2R7  T2R9  T3R1 
15731  7981  8945  5096  6864  6832    54   371    63   104   487  2890    80 
T3R10  T3R5  T3R7  T3R9  T4R1 T4R10  T4R7  T4R9  T5R1 T5R10  T5R4  T5R5  T5R7 
 1247    18   117  2514    26   488   111  1000    90   918  1725   726    39 
 T5R9 
 3953

In [11]:
physeq.whole.genus.prevalent.rel <- transform_sample_counts(physeq.whole.genus.prevalent, function(x) 100 * x/sum(x))

In [12]:
sample_sums(physeq.whole.genus.prevalent.rel)

T1R1 T1R10  T1R4  T1R5  T1R7  T1R9  T2R1 T2R10  T2R4  T2R5  T2R7  T2R9  T3R1 
  100   100   100   100   100   100   100   100   100   100   100   100   100 
T3R10  T3R5  T3R7  T3R9  T4R1 T4R10  T4R7  T4R9  T5R1 T5R10  T5R4  T5R5  T5R7 
  100   100   100   100   100   100   100   100   100   100   100   100   100 
 T5R9 
  100

### MDF

In [67]:
# to reduce the name length: p.rel.mdf = physeq.whole.genus.prevelant.rel.mdf
p.rel.mdf = subset_samples(physeq.whole.genus.prevalent.rel, Material == "MDF")

In [68]:
# filter out features that are not present in all samples
p.rel.mdf = prune_taxa(taxa_sums(p.rel.mdf) > 0, p.rel.mdf)
# filter out fatures that are NA at Genus level
p.rel.mdf = subset_taxa(p.rel.mdf, !Genus == "g__")

In [69]:
p.rel.mdf

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 17 taxa and 12 samples ]
sample_data() Sample Data:       [ 12 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 17 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 17 tips and 16 internal nodes ]

In [70]:
# log10(x+1) transformation
rel.mdf = otu_table(p.rel.mdf)
rel.mdf = log10(rel.mdf + 1)

In [71]:
# combine feature table with taxonomy
rel.mdf.w.taxa = cbind(rel.mdf, tax_table(p.rel.mdf))

In [72]:
rel.mdf.w.taxa

,T1R1,T1R4,T1R5,T2R1,T2R4,T2R5,T3R1,T3R5,T4R1,T5R1,T5R4,T5R5,Kingdom,Phylum,Class,Order,Family,Genus,Species
ac09a700281e0e84b375bb5ae517a23e,1.18495605793315,1.15135049083184,0.610750389434248,0,0,0,1.65079303965193,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Geobacillus,NA
896c05c0ef629cc4725eb1d8db95f2e5,0,0,0,0,0,0,0,0,1.93255181202787,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Bacillus,NA
4921e77373cf88d5d939898928db7f1d,0.599339042945075,0.425816981866494,0.70432294893801,0,0,1.5632071689666,0,1.53571596998551,0,0,1.04933040318107,1.01562759328375,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Staphylococcaceae,g__Staphylococcus,NA
af2a8d6190dac41427b5208bd79874e0,0,0,0,0,0,0,0,0,0,1.42415539150881,1.53203392604035,0.979545866513348,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Lactobacillus,NA
04b867faf28bb8e9696ddab9bc9774a1,0,0.789137531155893,0.18464241111829,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,NA
3112a0cc9050d0bee3e65ae6ac080099,0,0.720008998492251,0.49678197613419,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Lactococcus,NA
6ba2d1f0e3b971b2afeb348b44f26b2d,0,0.559567433591381,0,0,0,0,0,0,0,0,0.535899255272849,0.350183920557956,k__Bacteria,p__Firmicutes,c__Clostridia,o__Clostridiales,f__[Tissierellaceae],g__Anaerococcus,NA
2424b82f393cce6b25e2f0fef86314f2,1.13916470377843,1.18050315700234,0.699242631138382,1.63941269868445,0,1.23920319390714,1.41497334797082,0,0,1.72609300397524,1.23340080687626,0.822650696872673,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas,NA
f508a05c1f9bbdaefd0ec0ff6f5db60b,0.387925354182618,0,0.426181736583728,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Proteobacteria,c__Betaproteobacteria,o__Burkholderiales,f__Oxalobacteraceae,g__Massilia,NA
92eb684563a59988b1e9cc61e7c6eaa1,1.48595810129406,0.971378542002719,1.9015062230638,1.76646792916992,1.8875637201146,1.50200669934956,1.32221929473392,0,0,0.995147497205588,1.06478342534698,1.64184543774214,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Moraxellaceae,g__Acinetobacter,NA


In [73]:
# Pearson correlation
cor.rel.mdf = rcorr(t(rel.mdf.w.taxa)[1:12,], type='pearson')

In [77]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.m = which(abs(cor.rel.mdf$r) > 0.3 & cor.rel.mdf$P < 0.05 & lower.tri(cor.rel.mdf$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.m.by.row = indices.m[order(indices.m[,1]),]

sig.rel.mdf = cor.rel.mdf$r[indices.m.by.row]
length(sig.rel.mdf)

[1] 10

In [78]:
sig.rel.mdf

[1]  0.9215014 -0.5862198  0.7475874  0.5949603  0.9524653  0.6142225
 [7]  0.6255170  0.6353404  0.8039374  0.9700312

In [79]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.rel.mdf$r[which(abs(cor.rel.mdf$r) <= 0.3 & lower.tri(cor.rel.mdf$r), arr.ind=TRUE)]=0
cor.rel.mdf$r[which(cor.rel.mdf$P >= 0.05 & lower.tri(cor.rel.mdf$r), arr.ind=TRUE)]=0

In [80]:
# show vertex label in Genus level
v.names.r.m <- rel.mdf.w.taxa[,18]

In [81]:
is.vector(v.names.r.m)

[1] TRUE

In [82]:
unique(rel.mdf.w.taxa[,16])

[1] "o__Bacillales"       "o__Lactobacillales"  "o__Clostridiales"   
 [4] "o__Pseudomonadales"  "o__Burkholderiales"  "o__Deinococcales"   
 [7] "o__Thermales"        "o__Actinomycetales"  "o__Sphingomonadales"
[10] "o__Caulobacterales"  "o__Rhizobiales"

In [83]:
# color vertex according to Order level
colourCount = length(unique(rel.mdf.w.taxa[,16]))
getPalette = colorRampPalette(brewer.pal(8, "Set2"))
col.r.m = getPalette(colourCount)
my_color.r.m = col.r.m[as.numeric(as.factor(rel.mdf.w.taxa[,15]))]

In [85]:
# create network graph from correlation matrix
nw.rel.mdf <- graph_from_adjacency_matrix(cor.rel.mdf$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.rel.mdf)$name <- v.names.r.m
V(nw.rel.mdf)$size <- degree(nw.rel.mdf)
E(nw.rel.mdf)$values <- sig.rel.mdf
# red for negative correlation, green for positive correlation
E(nw.rel.mdf)$color <- ifelse(E(nw.rel.mdf)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('rel_mdf_correlation_prevalent.pdf', width=20, height=20)
l.r.m <- layout_with_graphopt(nw.rel.mdf)
plot.igraph(nw.rel.mdf,
            layout=l.r.m, 
            vertex.label.cex=0.9,
            vertex.color=my_color.r.m)
legend(x="topleft", legend=levels(as.factor(rel.mdf.w.taxa[,16])), col = col.r.m, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

### Gypsum

In [13]:
p.rel.gyp = subset_samples(physeq.whole.genus.prevalent.rel, Material == "Gypsum")

In [14]:
p.rel.gyp = prune_taxa(taxa_sums(p.rel.gyp) > 0, p.rel.gyp)
p.rel.gyp = subset_taxa(p.rel.gyp, !Genus == "g__")

In [15]:
p.rel.gyp

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 19 taxa and 15 samples ]
sample_data() Sample Data:       [ 15 samples by 10 sample variables ]
tax_table()   Taxonomy Table:    [ 19 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 19 tips and 18 internal nodes ]

In [16]:
# log10(x+1) transformation
rel.gyp <- otu_table(p.rel.gyp)
rel.gyp <- log10(rel.gyp + 1)

In [17]:
# combine feature table with taxonomy
rel.gyp.w.taxa <- cbind(rel.gyp, tax_table(p.rel.gyp))

In [18]:
rel.gyp.w.taxa

,T1R10,T1R7,T1R9,T2R10,T2R7,T2R9,T3R10,T3R7,T3R9,T4R10,⋯,T5R10,T5R7,T5R9,Kingdom,Phylum,Class,Order,Family,Genus,Species
ac09a700281e0e84b375bb5ae517a23e,0.505860206894496,0.798544409025487,0.9028992420294,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Geobacillus,NA
896c05c0ef629cc4725eb1d8db95f2e5,0,0,0.672389900610418,2.00432137378264,2.00432137378264,1.85881185153143,1.69334043538589,2.00432137378264,1.60186128292692,1.43829588168303,⋯,1.26293166694269,1.84652742522746,1.37366643318604,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Bacillus,NA
473b630262c9e46dc6f8f08886406faf,0,0,0,0,0,0.430639614916017,0,0,0.559343750896491,0,⋯,0.383118860611899,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Paenibacillaceae,g__Paenibacillus,NA
4921e77373cf88d5d939898928db7f1d,1.22982108275077,0,0.878371090378881,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Staphylococcaceae,g__Staphylococcus,NA
af2a8d6190dac41427b5208bd79874e0,0,0.625281263967879,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Lactobacillus,NA
04b867faf28bb8e9696ddab9bc9774a1,0,0,0.862088067296263,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,NA
3112a0cc9050d0bee3e65ae6ac080099,0,0.326703395421257,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Lactococcus,NA
2424b82f393cce6b25e2f0fef86314f2,1.51020326392271,1.28748329967158,1.49997022862778,0,0,0.824434384887337,0,0,1.10902544729242,0,⋯,0,0,0.924904017993072,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas,NA
f508a05c1f9bbdaefd0ec0ff6f5db60b,0.165422713123016,0,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Proteobacteria,c__Betaproteobacteria,o__Burkholderiales,f__Oxalobacteraceae,g__Massilia,NA
92eb684563a59988b1e9cc61e7c6eaa1,0,1.37922942696791,1.41910993429852,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Moraxellaceae,g__Acinetobacter,NA


In [19]:
# Pearson correlation
cor.rel.gyp <- rcorr(t(rel.gyp.w.taxa)[1:15,], type='pearson')

Warning message in sqrt(1 - h * h):
“NaNs produced”

In [52]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices = which(abs(cor.rel.gyp$r) > 0.3 & cor.rel.gyp$P < 0.05 & lower.tri(cor.rel.gyp$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.by.row = indices[order(indices[,1]),]

sig.rel.gyp = cor.rel.gyp$r[indices.by.row]
length(sig.rel.gyp)

[1] 64

In [53]:
sig.rel.gyp

[1] -0.8394510  0.6662085 -0.6659563  0.5731364 -0.5769913  0.6649547
 [7]  0.5417012  0.5731364 -0.5769913  1.0000000  0.7560412 -0.7450674
[13]  0.6536792 -0.5769913  0.7997311  0.9093641 -0.6395048  0.6701019
[19]  0.6925087  0.6701019  0.5925581  0.6748804 -0.8261015  0.8255442
[25]  0.8255442  0.5657145  0.5209764  0.5616112 -0.8022901  0.6815884
[31]  0.5850924  0.8797289  0.8540103  0.5731364 -0.5769913  1.0000000
[37]  0.6701019  0.8255442  0.9287008 -0.9228112  0.7728770  0.5504372
[43]  0.5504372  0.7853464  0.6172196  0.7105789  0.8258672  0.8263916
[49]  0.5504372  0.5279766 -0.5769913  0.7997311  0.8797289  0.6172196
[55]  0.9102421 -0.6203698  0.6051491  0.7508537  0.6051491  0.5950663
[61]  0.9964410  0.6051491  0.6996150  0.7454537

In [54]:
#which(cor.rel.gyp$P == 'NaN')

In [55]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.rel.gyp$r[which(abs(cor.rel.gyp$r) <= 0.3 & lower.tri(cor.rel.gyp$r), arr.ind=TRUE)]=0
cor.rel.gyp$r[which(cor.rel.gyp$P >= 0.05 & lower.tri(cor.rel.gyp$r), arr.ind=TRUE)]=0

In [57]:
cor.rel.gyp$r[which(cor.rel.gyp$P == 'NaN' & lower.tri(cor.rel.gyp$r), arr.ind=TRUE)]=0

In [58]:
cor.rel.gyp$r

,ac09a700281e0e84b375bb5ae517a23e,896c05c0ef629cc4725eb1d8db95f2e5,473b630262c9e46dc6f8f08886406faf,4921e77373cf88d5d939898928db7f1d,af2a8d6190dac41427b5208bd79874e0,04b867faf28bb8e9696ddab9bc9774a1,3112a0cc9050d0bee3e65ae6ac080099,2424b82f393cce6b25e2f0fef86314f2,f508a05c1f9bbdaefd0ec0ff6f5db60b,92eb684563a59988b1e9cc61e7c6eaa1,157f6ba228fbb1d6d88977bcc75d3c63,4c6043796448a2d50aaf470a409f36f7,04f7a79aa778df478752a5e08bb5e68d,7e2c443c0ab5d9675d2e14bf13cdbab5,ba2bf0826df666849c91fc7512a57e78,0c00cc21bf4cb3fbc403e974cad42402,d1589540ef4c33b985e1103a5e0921f9,178902d35f055b7cc539fe7326a49c89,8f1d6c5f6a03b21a67f492d2004679fa
ac09a700281e0e84b375bb5ae517a23e,1.0000000,-0.8394510,-0.2383179,0.6662085,0.5731364,0.66495466,0.57313643,0.7560412,0.31561364,0.9093641,0.6748804,0.5616112,0.57313643,-0.24151318,0.9287008,-0.43222452,0.31561364,0.9102421,-0.31731249
896c05c0ef629cc4725eb1d8db95f2e5,-0.8394510,1.0000000,0.1481678,-0.6659563,-0.5769913,-0.29857671,-0.57699133,-0.7450674,-0.57699133,-0.6395048,-0.8261015,-0.8022901,-0.57699133,0.08445979,-0.9228112,0.13910727,-0.57699133,-0.6203698,-0.02763061
473b630262c9e46dc6f8f08886406faf,0.0000000,0.0000000,1.0000000,-0.1899306,-0.1314745,-0.13147447,-0.13147447,0.1888296,-0.13147447,-0.1929292,-0.1882372,-0.1828115,-0.13147447,-0.24524210,-0.2284842,0.52797661,-0.13147447,-0.1919936,0.40278309
4921e77373cf88d5d939898928db7f1d,0.6662085,-0.6659563,0.0000000,1.0000000,-0.1031871,0.54170122,-0.10318710,0.6536792,0.79973111,0.3284870,0.3631183,0.6815884,-0.10318710,-0.19247708,0.7728770,-0.34446696,0.79973111,0.3640329,-0.08047949
af2a8d6190dac41427b5208bd79874e0,0.5731364,-0.5769913,0.0000000,0.0000000,1.0000000,-0.07142857,1.00000000,0.3564164,-0.07142857,0.6701019,0.8255442,0.4114233,1.00000000,-0.13323722,0.5504372,-0.23844824,-0.07142857,0.6051491,-0.20502385
04b867faf28bb8e9696ddab9bc9774a1,0.6649547,0.0000000,0.0000000,0.5417012,0.0000000,1.00000000,-0.07142857,0.4498670,-0.07142857,0.6925087,-0.1022671,-0.0993194,-0.07142857,-0.13323722,0.4198924,-0.23844824,-0.07142857,0.7508537,-0.32194829
3112a0cc9050d0bee3e65ae6ac080099,0.5731364,-0.5769913,0.0000000,0.0000000,1.0000000,0.00000000,1.00000000,0.3564164,-0.07142857,0.6701019,0.8255442,0.4114233,1.00000000,-0.13323722,0.5504372,-0.23844824,-0.07142857,0.6051491,-0.20502385
2424b82f393cce6b25e2f0fef86314f2,0.7560412,-0.7450674,0.0000000,0.6536792,0.0000000,0.00000000,0.00000000,1.0000000,0.45436742,0.5925581,0.5657145,0.5850924,0.35641635,-0.11519923,0.7853464,-0.20786566,0.45436742,0.5950663,-0.10766093
f508a05c1f9bbdaefd0ec0ff6f5db60b,0.0000000,-0.5769913,0.0000000,0.7997311,0.0000000,0.00000000,0.00000000,0.0000000,1.00000000,-0.1048162,0.5039285,0.8797289,-0.07142857,-0.13323722,0.6172196,-0.23844824,1.00000000,-0.1043079,0.13444481
92eb684563a59988b1e9cc61e7c6eaa1,0.9093641,-0.6395048,0.0000000,0.0000000,0.6701019,0.69250872,0.67010191,0.5925581,0.00000000,1.0000000,0.5209764,0.2236543,0.67010191,-0.19551587,0.7105789,-0.34990535,-0.10481620,0.9964410,-0.38786915


In [59]:
# show vertex label in Genus level
v.names.r.g <- rel.gyp.w.taxa[,21]

In [60]:
# color vertex according to Order level
colourCount.g = length(unique(rel.gyp.w.taxa[,19]))
getPalette.g = colorRampPalette(brewer.pal(8, "Set2"))
col.r.g = getPalette.g(colourCount.g)
my_color.r.g = col.r.g[as.numeric(as.factor(rel.gyp.w.taxa[,19]))]

In [66]:
# create network graph from correlation matrix
nw.rel.gyp <- graph_from_adjacency_matrix(cor.rel.gyp$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.rel.gyp)$name <- v.names.r.g
V(nw.rel.gyp)$size <- degree(nw.rel.gyp)
E(nw.rel.gyp)$values <- sig.rel.gyp
# red for negative correlation, green for positive correlation
E(nw.rel.gyp)$color <- ifelse(E(nw.rel.gyp)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(333)
pdf('rel_gyp_correlation_prevalent.pdf', width=20, height=20)
l.r.g <- layout_with_graphopt(nw.rel.gyp)
plot.igraph(nw.rel.gyp,
            layout=l.r.g, 
            vertex.label.cex=0.9,
            vertex.color=my_color.r.g)
legend(x="topleft", legend=levels(as.factor(rel.gyp.w.taxa[,19])), col = col.r.g, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Quantitative

### MDF

In [86]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.mdf)
count.mdf = as.data.frame(sample_data(p.rel.mdf))$Count
p.quan.mdf = p.rel.mdf
otu_table(p.quan.mdf) = rel_to_quan(p.quan.mdf, count.mdf)

In [87]:
otu_table(p.quan.mdf)

,T1R1,T1R4,T1R5,T2R1,T2R4,T2R5,T3R1,T3R5,T4R1,T5R1,T5R4,T5R5
ac09a700281e0e84b375bb5ae517a23e,19013.027,3422.4709,1015.4330,0.000,0.000,0.0000,41111.76,0.00,0.000,0.00,0.000,0.000
896c05c0ef629cc4725eb1d8db95f2e5,0.000,0.0000,0.0000,0.000,0.000,0.0000,0.00,0.00,33377.596,0.00,0.000,0.000
4921e77373cf88d5d939898928db7f1d,3952.953,432.8932,1338.8193,0.000,0.000,6860.9503,0.00,13819.08,0.000,0.00,22609.650,17734.512
af2a8d6190dac41427b5208bd79874e0,0.000,0.0000,0.0000,0.000,0.000,0.0000,0.00,0.00,0.000,128265.01,73224.436,16169.702
04b867faf28bb8e9696ddab9bc9774a1,0.000,1339.3541,174.6286,0.000,0.000,0.0000,0.00,0.00,0.000,0.00,0.000,0.000
3112a0cc9050d0bee3e65ae6ac080099,0.000,1104.0229,704.9822,0.000,0.000,0.0000,0.00,0.00,0.000,0.00,0.000,0.000
6ba2d1f0e3b971b2afeb348b44f26b2d,0.000,682.7510,0.0000,0.000,0.000,0.0000,0.00,0.00,0.000,0.00,5395.485,2347.215
2424b82f393cce6b25e2f0fef86314f2,16977.425,3678.1394,1319.4162,8462.817,0.000,3152.3285,23492.43,0.00,0.000,262106.77,35712.971,10692.868
f508a05c1f9bbdaefd0ec0ff6f5db60b,1917.351,0.0000,549.7567,0.000,0.000,0.0000,0.00,0.00,0.000,0.00,0.000,0.000
92eb684563a59988b1e9cc61e7c6eaa1,39352.151,2173.1819,25942.0503,11406.405,23542.524,5933.7949,18793.95,0.00,0.000,44613.92,23508.898,81109.314


In [88]:
# log10(x+1) transformation
quan.mdf <- otu_table(p.quan.mdf)
quan.mdf <- log10(quan.mdf + 1)

In [89]:
# combine feature table with taxonomy
quan.mdf.w.taxa <- cbind(quan.mdf, tax_table(p.quan.mdf))

In [90]:
quan.mdf.w.taxa

,T1R1,T1R4,T1R5,T2R1,T2R4,T2R5,T3R1,T3R5,T4R1,T5R1,T5R4,T5R5,Kingdom,Phylum,Class,Order,Family,Genus,Species
ac09a700281e0e84b375bb5ae517a23e,4.27907410099931,3.53446664620083,3.00707877043809,0,0,0,4.61397661538716,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Geobacillus,NA
896c05c0ef629cc4725eb1d8db95f2e5,0,0,0,0,0,0,0,0,4.52346806705087,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Bacillus,NA
4921e77373cf88d5d939898928db7f1d,3.59703146967911,2.63738282776358,3.1270462451606,0,0,3.83644756942982,0,4.14051050769951,0,0,4.35431305508605,4.24884374019123,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Staphylococcaceae,g__Staphylococcus,NA
af2a8d6190dac41427b5208bd79874e0,0,0,0,0,0,0,0,0,0,5.10811159436312,4.86466196596562,4.20872888671369,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Lactobacillus,NA
04b867faf28bb8e9696ddab9bc9774a1,0,3.12721954033693,2.2445952657756,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,NA
3112a0cc9050d0bee3e65ae6ac080099,0,3.04337127237323,2.84879373217879,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Lactococcus,NA
6ba2d1f0e3b971b2afeb348b44f26b2d,0,2.83489797115151,0,0,0,0,0,0,0,0,3.73211095447657,3.37073783499707,k__Bacteria,p__Firmicutes,c__Clostridia,o__Clostridiales,f__[Tissierellaceae],g__Anaerococcus,NA
2424b82f393cce6b25e2f0fef86314f2,4.22989740202698,3.56574624341188,3.12071083392782,3.92756625812147,0,3.49876921946465,4.37094648925104,0,0,5.4184798873132,4.55283813576996,4.029134809676,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas,NA
f508a05c1f9bbdaefd0ec0ff6f5db60b,3.28292807035542,0,2.74095981845948,0,0,0,0,0,0,0,0,0,k__Bacteria,p__Proteobacteria,c__Betaproteobacteria,o__Burkholderiales,f__Oxalobacteraceae,g__Massilia,NA
92eb684563a59988b1e9cc61e7c6eaa1,4.59497951106934,3.33729587384965,4.41402103722461,4.05718687395694,4.37187146515562,3.77340571267013,4.27404109766384,0,0,4.64948009383295,4.37125074228789,4.90907608294126,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Moraxellaceae,g__Acinetobacter,NA


In [91]:
# Pearson correlation
cor.quan.mdf <- rcorr(t(quan.mdf.w.taxa)[1:12,], type='pearson')

In [95]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.q.m = which(abs(cor.quan.mdf$r) > 0.3 & cor.quan.mdf$P < 0.05 & lower.tri(cor.quan.mdf$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.q.m.by.row = indices.q.m[order(indices.q.m[,1]),]

sig.quan.mdf = cor.quan.mdf$r[indices.q.m.by.row]
length(sig.quan.mdf)

[1] 12

In [96]:
sig.quan.mdf

[1]  0.9899458  0.5773652  0.5792659 -0.6545045  0.7407023  0.8004208
 [7]  0.7151052  0.7676832  0.7433873  0.5849469  0.9845169  0.8491220

In [97]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.quan.mdf$r[which(abs(cor.quan.mdf$r) <= 0.3 & lower.tri(cor.quan.mdf$r), arr.ind=TRUE)]=0
cor.quan.mdf$r[which(cor.quan.mdf$P >= 0.05 & lower.tri(cor.quan.mdf$r), arr.ind=TRUE)]=0

In [98]:
# show vertex label in Genus level
v.names.q.m <- quan.mdf.w.taxa[,18]

In [99]:
# color vertex according to Order level
colourCount.q.m = length(unique(quan.mdf.w.taxa[,16]))
getPalette.q.m = colorRampPalette(brewer.pal(8, "Set2"))
col.q.m = getPalette(colourCount.q.m)
my_color.q.m = col.q.m[as.numeric(as.factor(quan.mdf.w.taxa[,16]))]

In [103]:
# create network graph from correlation matrix
nw.quan.mdf <- graph_from_adjacency_matrix(cor.quan.mdf$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.quan.mdf)$name <- v.names.q.m
V(nw.quan.mdf)$size <- degree(nw.quan.mdf)
E(nw.quan.mdf)$values <- sig.quan.mdf
# red for negative correlation, green for positive correlation
E(nw.quan.mdf)$color <- ifelse(E(nw.quan.mdf)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(333)
pdf('quan_mdf_correlation_prevalent.pdf', width=20, height=20)
l.q.m <- layout_with_graphopt(nw.quan.mdf)
plot.igraph(nw.quan.mdf,
            layout=l.q.m, 
            vertex.label.cex=0.9,
            vertex.color=my_color.r.g)
legend(x="topleft", legend=levels(as.factor(quan.mdf.w.taxa[,16])), col = col.q.m, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

### Gypsum

In [104]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.gyp)
count.gyp = as.data.frame(sample_data(p.rel.gyp))$Count
p.quan.gyp = p.rel.gyp
otu_table(p.quan.gyp) = rel_to_quan(p.quan.gyp, count.gyp)

In [105]:
otu_table(p.quan.gyp)

,T1R10,T1R7,T1R9,T2R10,T2R7,T2R9,T3R10,T3R7,T3R9,T4R10,T4R7,T4R9,T5R10,T5R7,T5R9
ac09a700281e0e84b375bb5ae517a23e,10598.080,7788.106,7727.585,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
896c05c0ef629cc4725eb1d8db95f2e5,0.000,0.000,4090.123,204769.9,63732.76,360168.8043,412012.4,420760,186279.19,200576.6,331858.6,243266.6,140007.41,698137.9,201325.45
473b630262c9e46dc6f8f08886406faf,0.000,0.000,0.000,0.0,0.00,8571.2829,0.0,0,12545.33,0.0,0.0,0.0,11447.15,0.0,0.00
4921e77373cf88d5d939898928db7f1d,76775.866,0.000,7242.590,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
af2a8d6190dac41427b5208bd79874e0,0.000,4741.519,0.000,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
04b867faf28bb8e9696ddab9bc9774a1,0.000,0.000,6935.427,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
3112a0cc9050d0bee3e65ae6ac080099,0.000,1652.022,0.000,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
2424b82f393cce6b25e2f0fef86314f2,150781.779,27076.004,33820.309,0.0,0.00,28687.5590,0.0,0,56644.08,0.0,0.0,0.0,0.00,0.0,65908.78
f508a05c1f9bbdaefd0ec0ff6f5db60b,2228.006,0.000,0.000,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
92eb684563a59988b1e9cc61e7c6eaa1,0.000,33791.368,27887.205,0.0,0.00,0.0000,0.0,0,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [106]:
# log10(x+1) transformation
quan.gyp <- otu_table(p.quan.gyp)
quan.gyp <- log10(quan.gyp + 1)

In [107]:
# combine feature table with taxonomy
quan.gyp.w.taxa <- cbind(quan.gyp, tax_table(p.quan.gyp))

In [108]:
quan.gyp.w.taxa

,T1R10,T1R7,T1R9,T2R10,T2R7,T2R9,T3R10,T3R7,T3R9,T4R10,⋯,T5R10,T5R7,T5R9,Kingdom,Phylum,Class,Order,Family,Genus,Species
ac09a700281e0e84b375bb5ae517a23e,4.02526818308335,3.89148760121805,3.88809999001897,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Geobacillus,NA
896c05c0ef629cc4725eb1d8db95f2e5,0,0,3.61184258596143,5.3112681684589,4.80436957271632,5.55650729992711,5.61491138673002,5.62403547860355,5.27016666887256,5.30228251032664,⋯,5.14615413607238,5.84394186145231,5.3039008277277,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Bacillaceae,g__Bacillus,NA
473b630262c9e46dc6f8f08886406faf,0,0,0,0,0,3.93309649312652,0,0,4.09851681394266,0,⋯,4.0587351995758,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Paenibacillaceae,g__Paenibacillus,NA
4921e77373cf88d5d939898928db7f1d,4.88523037999031,0,3.85995387033102,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Bacillales,f__Staphylococcaceae,g__Staphylococcus,NA
af2a8d6190dac41427b5208bd79874e0,0,3.67600907393241,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Lactobacillaceae,g__Lactobacillus,NA
04b867faf28bb8e9696ddab9bc9774a1,0,0,3.84113580388991,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,NA
3112a0cc9050d0bee3e65ae6ac080099,0,3.2182787481391,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Lactococcus,NA
2424b82f393cce6b25e2f0fef86314f2,5.17835174341727,4.43260061038881,4.52919041773769,0,0,4.4577087340608,0,0,4.75316219285413,0,⋯,0,0,4.8189498448719,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas,NA
f508a05c1f9bbdaefd0ec0ff6f5db60b,3.34811114416627,0,0,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Proteobacteria,c__Betaproteobacteria,o__Burkholderiales,f__Oxalobacteraceae,g__Massilia,NA
92eb684563a59988b1e9cc61e7c6eaa1,0,4.52881862611453,4.44542056888128,0,0,0,0,0,0,0,⋯,0,0,0,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Moraxellaceae,g__Acinetobacter,NA


In [109]:
# Pearson correlation
cor.quan.gyp <- rcorr(t(quan.gyp.w.taxa)[1:15,], type='pearson')

In [113]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.q.g = which(abs(cor.quan.gyp$r) > 0.3 & cor.quan.gyp$P < 0.05 & lower.tri(cor.quan.gyp$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.q.g.by.row = indices.q.g[order(indices.q.g[,1]),]

sig.quan.gyp = cor.quan.gyp$r[indices.q.g.by.row]
length(sig.quan.gyp)

[1] 64

In [114]:
sig.quan.gyp

[1] -0.9069865  0.7855487 -0.6211117  0.5270555 -0.6563809  0.5264803
 [7]  0.5845928  0.5270555 -0.6563809  1.0000000  0.6158551 -0.5695684
[13]  0.5497676 -0.6563809  0.7674979  0.7730493 -0.5847443  0.6886572
[19]  0.6740452  0.6886572  0.7902917 -0.9632503  0.6736353  0.6736353
[25]  0.6890593  0.7841773 -0.9532729  0.5674919  0.5689396  0.5689396
[31]  0.7796402  0.9910487  0.5270555 -0.6563809  1.0000000  1.0000000
[37]  0.6886572  0.6736353  0.5689396  0.9149060 -0.8479805  0.7333054
[43]  0.7189403  0.5687014  0.6577120  0.7662302  0.7668733  0.5210570
[49]  0.5497676 -0.6563809  0.7674979  1.0000000  0.6890593  0.7796402
[55]  0.5687014  0.7730704 -0.5792845  0.6774043  0.6853459  0.6774043
[61]  0.9998813  0.6774043  0.6571730  0.7225858

In [115]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.quan.gyp$r[which(abs(cor.quan.gyp$r) <= 0.3 & lower.tri(cor.quan.gyp$r), arr.ind=TRUE)]=0
cor.quan.gyp$r[which(cor.quan.gyp$P >= 0.05 & lower.tri(cor.quan.gyp$r), arr.ind=TRUE)]=0

In [116]:
# show vertex label in Genus level
v.names.q.g <- quan.gyp.w.taxa[,21]

In [117]:
# color vertex according to Order level
colourCount.q.g = length(unique(quan.gyp.w.taxa[,19]))
getPalette.q.g = colorRampPalette(brewer.pal(8, "Set2"))
col.q.g = getPalette(colourCount.q.g)
my_color.q.g = col.q.g[as.numeric(as.factor(quan.gyp.w.taxa[,19]))]

In [119]:
# create network graph from correlation matrix
nw.quan.gyp <- graph_from_adjacency_matrix(cor.quan.gyp$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.quan.gyp)$name <- v.names.q.g
V(nw.quan.gyp)$size <- degree(nw.quan.gyp)
E(nw.quan.gyp)$values <- sig.quan.gyp
# red for negative correlation, green for positive correlation
E(nw.quan.gyp)$color <- ifelse(E(nw.quan.gyp)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('quan_gyp_correlation_prevalent.pdf', width=20, height=20)
l.q.g <- layout_with_graphopt(nw.quan.gyp)
plot.igraph(nw.quan.gyp,
            layout=l.q.g, 
            vertex.label.cex=0.9,
            vertex.color=my_color.q.g)
legend(x="topleft", legend=levels(as.factor(quan.gyp.w.taxa[,19])), col = col.q.g, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Correlation Matrix Visualization, RMP VS QMP

### MDF (17 taxa)

In [56]:
cor.rel.mdf$r

,ac09a700281e0e84b375bb5ae517a23e,896c05c0ef629cc4725eb1d8db95f2e5,4921e77373cf88d5d939898928db7f1d,af2a8d6190dac41427b5208bd79874e0,04b867faf28bb8e9696ddab9bc9774a1,3112a0cc9050d0bee3e65ae6ac080099,6ba2d1f0e3b971b2afeb348b44f26b2d,2424b82f393cce6b25e2f0fef86314f2,f508a05c1f9bbdaefd0ec0ff6f5db60b,92eb684563a59988b1e9cc61e7c6eaa1,157f6ba228fbb1d6d88977bcc75d3c63,4c6043796448a2d50aaf470a409f36f7,04f7a79aa778df478752a5e08bb5e68d,ba2bf0826df666849c91fc7512a57e78,d1589540ef4c33b985e1103a5e0921f9,178902d35f055b7cc539fe7326a49c89,8f1d6c5f6a03b21a67f492d2004679fa
ac09a700281e0e84b375bb5ae517a23e,1,-0.1984608,-0.2502302,-0.37187715,0.42290974,0.41236963,0.0604845,0.29113302,0.38455233,0.14992771,0.210359097,0.39430960,-0.15848300,-0.19006473,-0.19894621,0.41530600,0.440835032
896c05c0ef629cc4725eb1d8db95f2e5,0,1.0000000,-0.2995697,-0.17034607,-0.11150164,-0.13219687,-0.1698555,-0.46430987,-0.13466167,-0.58621984,-0.133116680,-0.20139300,0.95246533,-0.21434271,-0.19791328,-0.09090909,-0.112382091
4921e77373cf88d5d939898928db7f1d,0,0.0000000,1.0000000,0.08481192,-0.06128907,-0.02648373,0.2195986,-0.15087874,0.06168298,-0.13312363,0.256773112,-0.23005581,-0.27294154,0.61422254,0.15890456,0.01298119,0.029405477
af2a8d6190dac41427b5208bd79874e0,0,0.0000000,0.0000000,1.00000000,-0.20893253,-0.24771140,0.4631055,0.36219949,-0.25232996,-0.02571729,-0.249434942,0.13975818,-0.21909731,0.31151859,-0.19173839,-0.17034607,-0.210582329
04b867faf28bb8e9696ddab9bc9774a1,0,0.0000000,0.0000000,0.00000000,1.00000000,0.92150138,0.5758493,0.10144530,0.03157071,-0.03787950,0.747587415,0.49845519,-0.06579965,0.22315400,0.18110404,-0.11150164,-0.075936374
3112a0cc9050d0bee3e65ae6ac080099,0,0.0000000,0.0000000,0.00000000,0.92150138,1.00000000,0.4318707,0.04342551,0.30640917,0.09818769,0.594960271,0.47602775,0.02810024,0.13459673,0.07912893,-0.13219687,-0.005397162
6ba2d1f0e3b971b2afeb348b44f26b2d,0,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,1.0000000,0.17722398,-0.25160335,-0.03901845,0.414027793,0.11284500,-0.21846640,0.62551701,0.11243421,-0.16985555,-0.209975938
2424b82f393cce6b25e2f0fef86314f2,0,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,1.00000000,-0.01166905,0.38689086,0.206617765,0.39394214,-0.48976610,-0.25127618,-0.22306828,0.10777742,0.079330266
f508a05c1f9bbdaefd0ec0ff6f5db60b,0,0.0000000,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,1.00000000,0.35256932,-0.197182861,0.41369010,0.08558363,-0.26708228,-0.29316466,0.63534037,0.803937358
92eb684563a59988b1e9cc61e7c6eaa1,0,-0.5862198,0.0000000,0.00000000,0.00000000,0.00000000,0.0000000,0.00000000,0.00000000,1.00000000,-0.009903294,0.03174828,-0.47253562,-0.39212017,0.37385266,0.13276124,0.213177390


In [57]:
cor.quan.mdf$r

,ac09a700281e0e84b375bb5ae517a23e,896c05c0ef629cc4725eb1d8db95f2e5,4921e77373cf88d5d939898928db7f1d,af2a8d6190dac41427b5208bd79874e0,04b867faf28bb8e9696ddab9bc9774a1,3112a0cc9050d0bee3e65ae6ac080099,6ba2d1f0e3b971b2afeb348b44f26b2d,2424b82f393cce6b25e2f0fef86314f2,f508a05c1f9bbdaefd0ec0ff6f5db60b,92eb684563a59988b1e9cc61e7c6eaa1,157f6ba228fbb1d6d88977bcc75d3c63,4c6043796448a2d50aaf470a409f36f7,04f7a79aa778df478752a5e08bb5e68d,ba2bf0826df666849c91fc7512a57e78,d1589540ef4c33b985e1103a5e0921f9,178902d35f055b7cc539fe7326a49c89,8f1d6c5f6a03b21a67f492d2004679fa
ac09a700281e0e84b375bb5ae517a23e,1.0000000,-0.2090874,0.01792438,-0.3986598,0.4812215,0.4802929,-0.0858159,0.31634210,0.5792659,0.25903329,0.143578712,0.453195562,-0.01041308,-0.141127303,-0.20070345,0.48651919,0.5849469
896c05c0ef629cc4725eb1d8db95f2e5,0.0000000,1.0000000,-0.34648322,-0.1733332,-0.1327075,-0.1347518,-0.1726584,-0.49837551,-0.1341898,-0.65450449,-0.134492759,-0.207168181,0.80042077,-0.241872240,-0.21177920,-0.09090909,-0.1297939
4921e77373cf88d5d939898928db7f1d,0.0000000,0.0000000,1.00000000,0.1811581,0.1591421,0.1692297,0.5074260,0.12865906,0.2889557,0.02910982,0.245595311,-0.005643679,-0.20317015,0.715105171,0.17889225,0.23003275,0.2884723
af2a8d6190dac41427b5208bd79874e0,0.0000000,0.0000000,0.00000000,1.0000000,-0.2530288,-0.2569267,0.5773652,0.51237372,-0.2558551,0.37421826,-0.256432700,0.149672198,-0.25255913,0.389884213,-0.01744327,-0.17333322,-0.2474735
04b867faf28bb8e9696ddab9bc9774a1,0.0000000,0.0000000,0.00000000,0.0000000,1.0000000,0.9899458,0.2502341,0.07615987,0.2510341,0.06020132,0.534447447,0.454788440,0.20717052,0.280708366,0.15293130,-0.13270751,0.1634616
3112a0cc9050d0bee3e65ae6ac080099,0.0000000,0.0000000,0.00000000,0.0000000,0.9899458,1.0000000,0.1965812,0.07027955,0.3261921,0.08037483,0.459010979,0.452561754,0.27425335,0.265837028,0.10238156,-0.13475183,0.2222778
6ba2d1f0e3b971b2afeb348b44f26b2d,0.0000000,0.0000000,0.00000000,0.5773652,0.0000000,0.0000000,1.0000000,0.31992953,-0.2548590,0.24284177,0.210462337,-0.090948132,-0.25157586,0.767683210,0.33221329,-0.17265840,-0.2465100
2424b82f393cce6b25e2f0fef86314f2,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,1.00000000,0.1600633,0.74070232,0.114457222,0.443083122,-0.42018067,-0.008046516,-0.13709954,0.19064954,0.1763652
f508a05c1f9bbdaefd0ec0ff6f5db60b,0.5792659,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,0.00000000,1.0000000,0.25759182,-0.198523154,0.545346524,0.24551096,-0.142557991,-0.31260474,0.74338733,0.9845169
92eb684563a59988b1e9cc61e7c6eaa1,0.0000000,-0.6545045,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,0.74070232,0.0000000,1.00000000,-0.005892045,0.307049790,-0.46555666,-0.122373614,0.25089523,0.18963620,0.2531175


In [58]:
mdf.rel.quan = lowerUpper(t(cor.rel.mdf$r), cor.quan.mdf$r)

In [59]:
mdf.rel.quan

,ac09a700281e0e84b375bb5ae517a23e,896c05c0ef629cc4725eb1d8db95f2e5,4921e77373cf88d5d939898928db7f1d,af2a8d6190dac41427b5208bd79874e0,04b867faf28bb8e9696ddab9bc9774a1,3112a0cc9050d0bee3e65ae6ac080099,6ba2d1f0e3b971b2afeb348b44f26b2d,2424b82f393cce6b25e2f0fef86314f2,f508a05c1f9bbdaefd0ec0ff6f5db60b,92eb684563a59988b1e9cc61e7c6eaa1,157f6ba228fbb1d6d88977bcc75d3c63,4c6043796448a2d50aaf470a409f36f7,04f7a79aa778df478752a5e08bb5e68d,ba2bf0826df666849c91fc7512a57e78,d1589540ef4c33b985e1103a5e0921f9,178902d35f055b7cc539fe7326a49c89,8f1d6c5f6a03b21a67f492d2004679fa
ac09a700281e0e84b375bb5ae517a23e,NA,0.0000000,0.0000000,0,0.0000000,0.0000000,0.0000000,0,0.5792659,0.0000000,0,0,0.0000000,0.0000000,0,0.0000000,0.5849469
896c05c0ef629cc4725eb1d8db95f2e5,0,NA,0.0000000,0,0.0000000,0.0000000,0.0000000,0,0.0000000,-0.6545045,0,0,0.8004208,0.0000000,0,0.0000000,0.0000000
4921e77373cf88d5d939898928db7f1d,0,0.0000000,NA,0,0.0000000,0.0000000,0.0000000,0,0.0000000,0.0000000,0,0,0.0000000,0.7151052,0,0.0000000,0.0000000
af2a8d6190dac41427b5208bd79874e0,0,0.0000000,0.0000000,NA,0.0000000,0.0000000,0.5773652,0,0.0000000,0.0000000,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000
04b867faf28bb8e9696ddab9bc9774a1,0,0.0000000,0.0000000,0,NA,0.9899458,0.0000000,0,0.0000000,0.0000000,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000
3112a0cc9050d0bee3e65ae6ac080099,0,0.0000000,0.0000000,0,0.9215014,NA,0.0000000,0,0.0000000,0.0000000,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000
6ba2d1f0e3b971b2afeb348b44f26b2d,0,0.0000000,0.0000000,0,0.0000000,0.0000000,NA,0,0.0000000,0.0000000,0,0,0.0000000,0.7676832,0,0.0000000,0.0000000
2424b82f393cce6b25e2f0fef86314f2,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0.0000000,NA,0.0000000,0.7407023,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000
f508a05c1f9bbdaefd0ec0ff6f5db60b,0,0.0000000,0.0000000,0,0.0000000,0.0000000,0.0000000,0,NA,0.0000000,0,0,0.0000000,0.0000000,0,0.7433873,0.9845169
92eb684563a59988b1e9cc61e7c6eaa1,0,-0.5862198,0.0000000,0,0.0000000,0.0000000,0.0000000,0,0.0000000,NA,0,0,0.0000000,0.0000000,0,0.0000000,0.0000000


In [60]:
label.genus = tax_table(p.rel.mdf)[,6]
label.genus = as.character(label.genus)
length(label.genus)

[1] 17

In [61]:
rownames(mdf.rel.quan) = label.genus
colnames(mdf.rel.quan) = label.genus

In [63]:
pdf('mdf_correlation_matrix.pdf', width=30, height=30)
corrplot(mdf.rel.quan, method="circle", tl.col="black", na.label="square", na.label.col="gray", tl.srt=45, tl.cex=2.0)
dev.off()

pdf 
  2

### GYP (19 taxa)

In [64]:
gyp.rel.quan = lowerUpper(t(cor.rel.gyp$r), cor.quan.gyp$r)

In [65]:
gyp.rel.quan

,ac09a700281e0e84b375bb5ae517a23e,896c05c0ef629cc4725eb1d8db95f2e5,473b630262c9e46dc6f8f08886406faf,4921e77373cf88d5d939898928db7f1d,af2a8d6190dac41427b5208bd79874e0,04b867faf28bb8e9696ddab9bc9774a1,3112a0cc9050d0bee3e65ae6ac080099,2424b82f393cce6b25e2f0fef86314f2,f508a05c1f9bbdaefd0ec0ff6f5db60b,92eb684563a59988b1e9cc61e7c6eaa1,157f6ba228fbb1d6d88977bcc75d3c63,4c6043796448a2d50aaf470a409f36f7,04f7a79aa778df478752a5e08bb5e68d,7e2c443c0ab5d9675d2e14bf13cdbab5,ba2bf0826df666849c91fc7512a57e78,0c00cc21bf4cb3fbc403e974cad42402,d1589540ef4c33b985e1103a5e0921f9,178902d35f055b7cc539fe7326a49c89,8f1d6c5f6a03b21a67f492d2004679fa
ac09a700281e0e84b375bb5ae517a23e,NA,-0.9069865,0.0000000,0.7855487,0.5270555,0.5264803,0.5270555,0.6158551,0.5497676,0.7730493,0.7902917,0.7841773,0.5270555,0,0.9149060,0.0000000,0.5497676,0.7730704,0.0000000
896c05c0ef629cc4725eb1d8db95f2e5,-0.8394510,NA,0.0000000,-0.6211117,-0.6563809,0.0000000,-0.6563809,-0.5695684,-0.6563809,-0.5847443,-0.9632503,-0.9532729,-0.6563809,0,-0.8479805,0.0000000,-0.6563809,-0.5792845,0.0000000
473b630262c9e46dc6f8f08886406faf,0.0000000,0.0000000,NA,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.0000000,0.5210570,0.0000000,0.0000000,0.0000000
4921e77373cf88d5d939898928db7f1d,0.6662085,-0.6659563,0.0000000,NA,0.0000000,0.5845928,0.0000000,0.0000000,0.7674979,0.0000000,0.0000000,0.5674919,0.0000000,0,0.7333054,0.0000000,0.7674979,0.0000000,0.0000000
af2a8d6190dac41427b5208bd79874e0,0.5731364,-0.5769913,0.0000000,0.0000000,NA,0.0000000,1.0000000,0.0000000,0.0000000,0.6886572,0.6736353,0.5689396,1.0000000,0,0.0000000,0.0000000,0.0000000,0.6774043,0.0000000
04b867faf28bb8e9696ddab9bc9774a1,0.6649547,0.0000000,0.0000000,0.5417012,0.0000000,NA,0.0000000,0.0000000,0.0000000,0.6740452,0.0000000,0.0000000,0.0000000,0,0.0000000,0.0000000,0.0000000,0.6853459,0.0000000
3112a0cc9050d0bee3e65ae6ac080099,0.5731364,-0.5769913,0.0000000,0.0000000,1.0000000,0.0000000,NA,0.0000000,0.0000000,0.6886572,0.6736353,0.5689396,1.0000000,0,0.0000000,0.0000000,0.0000000,0.6774043,0.0000000
2424b82f393cce6b25e2f0fef86314f2,0.7560412,-0.7450674,0.0000000,0.6536792,0.0000000,0.0000000,0.0000000,NA,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0,0.7189403,0.0000000,0.0000000,0.0000000,0.0000000
f508a05c1f9bbdaefd0ec0ff6f5db60b,0.0000000,-0.5769913,0.0000000,0.7997311,0.0000000,0.0000000,0.0000000,0.0000000,NA,0.0000000,0.6890593,0.7796402,0.0000000,0,0.5687014,0.0000000,1.0000000,0.0000000,0.0000000
92eb684563a59988b1e9cc61e7c6eaa1,0.9093641,-0.6395048,0.0000000,0.0000000,0.6701019,0.6925087,0.6701019,0.5925581,0.0000000,NA,0.0000000,0.0000000,0.6886572,0,0.6577120,0.0000000,0.0000000,0.9998813,0.0000000


In [66]:
quan.label.genus = tax_table(p.rel.gyp)[,6]
quan.label.genus = as.character(quan.label.genus)

In [67]:
quan.label.genus

[1] "g__Geobacillus"       "g__Bacillus"          "g__Paenibacillus"    
 [4] "g__Staphylococcus"    "g__Lactobacillus"     "g__Streptococcus"    
 [7] "g__Lactococcus"       "g__Pseudomonas"       "g__Massilia"         
[10] "g__Acinetobacter"     "g__Deinococcus"       "g__Thermus"          
[13] "g__Meiothermus"       "g__Promicromonospora" "g__Corynebacterium"  
[16] "g__Sphingobium"       "g__Sphingomonas"      "g__Brevundimonas"    
[19] "g__Methylobacterium"

In [68]:
rownames(gyp.rel.quan) = quan.label.genus
colnames(gyp.rel.quan) = quan.label.genus

In [71]:
pdf('gyp_correlation_matrix.pdf', width=30, height=30)
corrplot(gyp.rel.quan, method="circle", tl.col="black", na.label="square", na.label.col="gray", tl.srt=45, tl.cex=2.0)
dev.off()

pdf 
  2